In [1]:
from rdflib import Graph, Literal
from rdflib.namespace import RDF, RDFS, SKOS
import rdflib
import os
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setTimeout(20)
schema = rdflib.Namespace("http://schema.org/")
owl = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
ocs = rdflib.Namespace("https://w3id.org/ocs/ont/")

ModuleNotFoundError: No module named 'rdflib'

In [2]:
from src.helpers import get_labels
from src.helpers import get_o

In [3]:
language = "en"
directory = r"../OpenCS/ontology/core/00/"

In [4]:
g = Graph()
g.bind("ocs", ocs)
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        new_node = Graph()
        new_node.parse(f)
        g += new_node
        

In [5]:
enitites_to_find = pd.DataFrame([(str(o), s) for s, p, o in g.triples((None, SKOS.closeMatch, None))], columns=["uri", "filename"])

In [6]:
names = ["(dbr:" + uri.split("/").pop() + ")" for uri in enitites_to_find['uri']]
names = [uri.split("/").pop()
             .replace("*", "\*")
             .replace("(", "\(")
             .replace(")", "\)")
             .replace(",", "\,")
             .replace("'", "\\'")
             .replace("-", "\-") for uri in enitites_to_find['uri']]
names = ["(dbr:" + uri + ")" for uri in names]

In [7]:
labels = []
for idx_start in range(0, len(names), 10):
    idx_end = idx_start + 10
    labels.append(get_labels(names[idx_start:idx_end], language))
labels_df = pd.concat(labels)

In [8]:
changed_enities = labels_df.merge(enitites_to_find, on = "uri", how="left")
changed_enities_names = [str(entity) for entity in changed_enities['filename']]

In [9]:
changed_enities

,uri,label,filename
0,http://dbpedia.org/resource/Algorithm,Algorithm,https://w3id.org/ocs/ont/C10
1,http://dbpedia.org/resource/Reconfigurability,Reconfigurability,https://w3id.org/ocs/ont/C112
2,http://dbpedia.org/resource/Crt0,Crt0,https://w3id.org/ocs/ont/C115
3,http://dbpedia.org/resource/Computer_science,Computer science,https://w3id.org/ocs/ont/C1
4,http://dbpedia.org/resource/Embedded_system,Embedded system,https://w3id.org/ocs/ont/C11
...,...,...,...
578,http://dbpedia.org/resource/Microsoft_Talisman,Microsoft Talisman,https://w3id.org/ocs/ont/C992
579,http://dbpedia.org/resource/Suppression_list,Suppression list,https://w3id.org/ocs/ont/C986
580,http://dbpedia.org/resource/Built-in_self-test,Built-in self-test,https://w3id.org/ocs/ont/C987
581,http://dbpedia.org/resource/Monitor_unit,Monitor unit,https://w3id.org/ocs/ont/C993


In [11]:
graph_triples ={}
for s, p, o in g.triples((None, None, None)):
    if str(s) in changed_enities_names:
        entity_name = s.split("/").pop()
        if entity_name not in graph_triples.keys():
            graph_triples[entity_name] = []
            #graph_triples[entity_name].bind("ocs", ocs)    
        graph_triples[entity_name].append((s, p, o))

In [13]:
graph_triples["C1"]

[(rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#related'),
  rdflib.term.URIRef('https://w3id.org/ocs/ont/C10')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#related'),
  rdflib.term.URIRef('https://w3id.org/ocs/ont/C19')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#prefLabel'),
  rdflib.term.Literal('Computer science', lang='en')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#related'),
  rdflib.term.URIRef('https://w3id.org/ocs/ont/C14')),
 (rdflib.term.URIRef('https://w3id.org/ocs/ont/C1'),
  rdflib.term.URIRef('http://www.w3.org/2

In [61]:
graph = {}
for key, value in graph_triples.items():
    val = value.copy()
    val.sort(key = get_o)
    g = Graph()
    g.bind("ocs", ocs)
    for triple in value:
        g.add((s, p, o))
    graph[key] = g

In [52]:
val = value.copy()
val.sort(key = get_o)
g = Graph()
g.bind("ocs", ocs)
for triple in value:
    g.add((s, p, o))

In [53]:
g

<Graph identifier=Na8d4136fa39245889108a4d07c24554c (<class 'rdflib.graph.Graph'>)>

In [58]:
for s, p, o in graph["C401"].triples((None, None, None)):
    print(s,p,o)

https://w3id.org/ocs/ont/C391 http://www.w3.org/2004/02/skos/core#related https://w3id.org/ocs/ont/C12580


In [36]:
for idx, row in changed_enities.iterrows():
    uri = row['filename'].split("/").pop()
    label = Literal(row['label'], lang = language)
    graph[uri].add((rdflib.term.URIRef(ocs[uri]), SKOS.prefLabel, label))

In [38]:
graph['C401'].serialize(destination = "test.ttl")

<Graph identifier=N5d3a263efcba44598459b9ab475b9102 (<class 'rdflib.graph.Graph'>)>

In [21]:
for key, value in graph.items():
    value.serialize(destination=directory+key+".ttl")

In [20]:
directory

'../OpenCS/ontology/core/00/'